In [5]:
import pandas as pd
import numpy as np

!pip install yfinance
import yfinance as yf

import plotly.express as px
import plotly.graph_objects as go


     |████████████████████████████████| 5.5MB 6.8MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=30680900d6d59404aa7adb9f068d040b8b3d1a007cb1ab4e907c3dc07ccff0d5
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


### **Mathematical Background:**

The covariance of an asset with a portfolio is the weighted average of asset's covariance to all the asset making up the portfolio.

$Cov(R_1,R_P) = Cov(R_1, w_1R_1 + w_2R_2) = w_1Cov(R_1,R_1) + w_2Cov(R_1,R_2) $

Portfolio variance is given by:
$ Var(R_p) = \sigma^2 = w'\Omega w $ ,  
where $\Omega$ is the covariance matrix of the return.
The **volatility** is given by the square root of the variance.

## **Key variables and dataset:**



### Data:

In [6]:
rf = 0.09 #risk-free rate at time of writing (3-month treasury bill rate)
Tickers = ['NOW', 'AMD', 'JPM', 'MSCI', 'GE', 'RMD', 'BIO', 'APD', 'REG', 'MPC', 'BKNG', '%5EGSPC'] #my portfolio tickers (arbitrary in SP500 list)
historical_data = yf.download(tickers= Tickers, period = "1y",group_by='ticker')

[*********************100%***********************]  12 of 12 completed


In [7]:
# Maching companies to adjusted return only
adj_close = pd.DataFrame()
for name in Tickers:
    adj_close[name] = historical_data[name]['Adj Close'] 

In [8]:
daily_returns = adj_close.pct_change() #daily returns


### Weights:

In [9]:
#equally weigthed portfolio
nb_stocks = len(Tickers)-1
weights = np.ones(nb_stocks)/nb_stocks
weights 

array([0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
       0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
       0.09090909])

## **Performances:**

In [10]:
# Normalize stock data based on initial price
def normalize(df):
  x = df.copy()
  for i in x.columns[1:]:
    x[i] = x[i]/x[i][0]
  return x

In [11]:
# Function to plot interactive plot
def interactive_plot(df, title):
  fig = px.line(title = title)
  for i in df.columns[1:]:
    fig.add_scatter(x = df.index, y = df[i], name = i)
  fig.show()

In [12]:
interactive_plot(normalize(adj_close), 'Normalized prices')

## **Mean-Variance model**

Markowitz's theory make 3 key assumptions:


*   Allocation is done by rational investors ( for the same return , they will choose the one with the lowest variance)
*   Capital markets are perfect
*   Returns are **normally distributed**"

Idealy, investors seek to reduce variance by diversifying their portfolio investments. Here, I will propose an arbitrary portfolio, made of SP500 companies, which is diversified across sectors (industries).

According to Markowitz, the level of investment in a particular financial asset should be based upon that assets' contribution to the distribution of the overall portfolio (covariability of assets' returns).

The efficient frontier represents the optimal retuns for each level of risk ($\sigma$).
Along the efficient frontier, the only way to achieve higher expected returns is by increasing the riskiness of the portfolio ($\sigma$).

A critical input to the mean-variance model is the estimated correlation between assets (even more if using ETF).



### Basic portfolio anlysis:

In [13]:
def portfolio_stats(Tickers, rf, weights):
    
    # Getting the data
    historical_data = yf.download(tickers= Tickers, period = "1y",group_by='ticker')
    
    returns = []
    volatilities = []

    df = pd.DataFrame()
    for name in Tickers:
      df[name] = historical_data[name]['Adj Close'].pct_change().dropna()
    
    # Generate associated returns and volatilities
    rp = np.sum(df.mean() * weights) * 252
    var_p = np.dot(weights.T, np.dot(df.cov() * 252, weights))
    std_p = np.sqrt(var_p)
    sharpe_p = (rp-rf)/std_p

    return rp, var_p, std_p, sharpe_p

In [14]:
Return, Variance, Volatility, Sharpe = portfolio_stats(Tickers[:-1], rf, weights)
print("Portfolio return:{}%".format(round(Return*100, 3)) +"\n"+
      "Portfolio variance:{}%".format(round(Variance*100,3)) +"\n"+
      "Portfolio volatility:{}%".format(round(Volatility*100, 3))+"\n"
      "Portfolio sharpe ratio:{}".format(Sharpe))

[*********************100%***********************]  11 of 11 completed
Portfolio return:34.749%
Portfolio variance:17.342%
Portfolio volatility:41.644%
Portfolio sharpe ratio:0.6183024090743083


### Simulation: (over n different allocation (weghts attribution)

In [15]:
def simulation(data, risk_free, iterations):

    returns = []
    volatilities = []
    sharpes = []  
    num_assets = len(df.columns)
    
    for i in range (iterations):
        # Generate random weights
        weights = np.random.dirichlet(np.ones(num_assets),size=1)[0]
        # Generate associated returns and volatilities
        r = data.mean() * weights * 252      
        returns.append(np.sum(r))       
        vol_p = np.sqrt(np.dot(weights.T, np.dot(data.cov() * 252, weights)))
        volatilities.append(vol_p)
        sharpes.append(r-risk_free/vol_p)
    
    # Convert lists to arrays
    returns = np.array(returns)
    volatilities = np.array(volatilities)
    sharpes = np.array(sharpes)
    
    return returns, volatilities, sharpes


In [16]:
num_assets = len(Tickers)
    
# Getting the data
historical_data = yf.download(tickers= Tickers, period = "1y",group_by='ticker')
    
df = pd.DataFrame()
for name in Tickers:
  df[name] = historical_data[name]['Adj Close'].pct_change().dropna()

df = df.drop('%5EGSPC', axis=1)

port_returns, port_vols, port_sharpes = simulation(df, rf, 10000)

[*********************100%***********************]  12 of 12 completed


### Efficient frontier:



We want to find the portfolios' weights that yield the Global Minimum Variance
Portfolio, such that the sum of the portfolio weights = 1. We can express this
problem as:

$ \min_{w } \sigma_p^2 $ such that:  $w'*e=1 $ (some of the weights equal to 1).

It turns out that the vector w that solves the problem above will also solve:

$ \min_{w } \frac{1}{2}\sigma_p^2 $ such that:  $w'*e=1 $. Notice that 1/2 is there to compensate the x2 that will happen when we procceed to tha Lagrangian Optimization equation.

$L = \frac{1}{2} w'\Omega w - \lambda_1(w'e) $ 

Using the constraint and calculate the derivative regarding w of L, gives the formula for the portfolios' weights that yield the Global Minimum Variance.

The optimal weights is given by the following formula:

$w* =\frac{\Omega^{-1}e}{e'\Omega^{-1}e} $

In [17]:
def optimal_weights(covariances, nb_assets):

    inv_cov = np.linalg.inv(covariances)
    Ones = np.ones(nb_assets) 
    weights = np.dot(inv_cov,Ones)/np.dot(Ones.T, np.dot(inv_cov, Ones))
    
    return weights

In [18]:
cov_matrice = daily_returns.drop('%5EGSPC', axis=1).cov()*252
nb_assets = len(Tickers[:-1])
optimal_w = optimal_weights(cov_matrice, nb_assets)

In [19]:
print(np.sum(optimal_w))

0.9999999999999998


Now let's say we have a vector of expected returns for all n assets. (Where’d we
get these expectations? Not now! We’ll get to that later.) The Expected Return
vector is $r$ and a portfolio’s expected return is the weighted average of the expected: 
$E(R_p) = r*w$.
To take into account the target, we have to add second constraint on $E(R_p) = T$ which results in the following Lagrangian optimization equation:

$L = \frac{1}{2} w'\Omega w - \lambda_1(w'e) - \lambda_2(w'r-T)$ 

Now by solving this optimization problem (derivative equal to 0 with regard to 0 + constraint), we get the following formula for the otpimal weights:

$w* = \frac{c-bT}{ac-b^2}\Omega^{-1}e + \frac{aT-b}{ac-b^2}\Omega^{-1}r$

where: 

$a=e'\Omega^{-1}e$

$b=e'\Omega^{-1}r$

$c=r'\Omega^{-1}r$


In [20]:
def optimal_weights_with_target(covariances, returns, nb_assets, Target):

  inv_cov = np.linalg.inv(covariances)
  Ones = np.ones(nb_assets) 

  a = np.dot(Ones.T, np.dot(inv_cov, Ones))
  b = np.dot(Ones.T, np.dot(inv_cov, returns))
  c = np.dot(returns.T, np.dot(inv_cov, returns))

  weights = ((c-b*Target)/(a*c-b**2))*np.dot(inv_cov,Ones) + ((a*Target-b)/(a*c-b**2))*np.dot(inv_cov,returns)

  return weights

In [21]:
cov_matrice = daily_returns.drop('%5EGSPC', axis=1).cov()*252
returns = daily_returns.drop('%5EGSPC', axis=1).mean()*252
nb_assets = len(Tickers[:-1])
optimal_w_T = optimal_weights_with_target(cov_matrice, returns, nb_assets, max(port_returns)) # just to check if it is working for the highest target randomly generated previously


In [22]:
print(np.sum(optimal_w_T))

1.0


To generate the **efficient frontier**, we just have to compute the variance for a large number of target returns.

In [23]:
def generate_min_volatilities(data, Targets):

    min_volatilities = []
    for target in Targets:
      # Calculate min volatilities for 100 portfolio returns (inside the max and min of the thousand previously randomly generated)
      cov = data.cov()*252
      opt_weights = optimal_weights_with_target(cov,data.mean()*252, len(data.columns), target)
      min_vol_p = np.sqrt(np.dot(opt_weights.T, np.dot(data.cov() * 252, opt_weights)))
      min_volatilities.append(min_vol_p)

    
    min_volatilities=np.array(min_volatilities)

    return min_volatilities
 

In [24]:
targets = np.linspace(port_returns.min(),port_returns.max(),100)
min_vols = generate_min_volatilities(daily_returns.drop('%5EGSPC', axis=1), targets)

In [25]:
def visualization_eficient_frontier(volatilities, returns, sharpes, min_volatilities, Targets):

    fig = go.Figure()
    fig.add_trace(go.Scatter(
    x = volatilities,
    y = returns ,
    mode='markers',
    marker=dict(
        size=16,
        color= (returns-rf)/volatilities, #set color equal to a variable
        colorscale='Viridis', # one of plotly colorscales
        showscale=True
      )
    ))

    fig.add_trace(go.Scatter(
    x = min_volatilities,
    y =  Targets,
    mode='lines',
    ))
    fig.update_layout(showlegend=False)
    fig.show()

In [26]:
visualization_eficient_frontier(port_vols, port_returns, port_sharpes, min_vols, targets)

It seems that the weights randomly generated are still far a way from the optimal weights. This is not surprising as we use a random distribution to do so with the ony constraint to have the weights that sum up to 1.

## **Capital Asset Pricing Model: (CAPM)**



CAPM shows the relation between the risk and expected return of a risky asset. The risky asset can be decomposed into 2 components: 


1.   Systematic risk = market risk (non-diversifiable)
2.   Specific risk = asset risk (diversifiable)

The CAPM demonstrates that by combining assets into a portfolio, each assets' unique risk can be eliminated. This leaves market risk as the portfolio's sole exposure.

CAPM requires solid asumptions:


*   Access to information for all participant equally
*   All participant make investment based on the mean and variance of returns (particulary false for retail traders by the way)
*   No frictions (taxes, cost etc ..) -> I wish
*   All participant can lend and borrow at a common risk free rate ( 3-month US treasury bond rate)
* Any individual investor's allocation cannot change the market prices

In this model, each investor's portfolio is  combination of assests specific risk and systematic risk.

Given an asset, its systematic risk is given by $\beta_i$ and is calculated as:

$\beta_i = \frac{cov(R_i, R_m)}{\sigma_m^2}$

It is very common to take the variance of the index for the market variance (ie: $\sigma_{SP500} = \sigma_m$) although subject to debate in the litterature.

Note that $\beta_i$, in practice, is estmated with a simple linea regression:

$r_{i,t} -r{t} = \alpha_{i} +\beta_{i}(R_{m,t}-r_t)+\epsilon_{i,t}$

The expected return over a given holding period on a given asset is:

$E(r_i) = rf + \beta_i[E(r_m-rf)]$

* E(rm)-r : market risk prenium

* $\beta_i$ : quantity of market risk

* rf = rate of return of the risk free asset (3-month treasury bond)

The tangent from risk free rate passing by a portfolio of the efficient frontier is called the capital market line (CML).
At the exception of  the contact point, the portfolios on the CML dominates (higher expected return) the portfolio on the efficient frontier.

Thus, all investors should allocate to 2 investments: 


*   the risk-free asset
*   the market portfolio

The equation for the CML is : 

$E(R_p) = rf + \frac{E(r_m - r)}{\sigma_m}\sigma_p$



### Market variables:

In [27]:
Rm = daily_returns['%5EGSPC'].mean()*252 #annual Market return (SP500 market return)
Sigma_m = daily_returns['%5EGSPC'].std()*np.sqrt(252)
print("The anualized market volatility is {}% and the annualized market  returns is {}%".format(round(Sigma_m*100,3),round(Rm*100,3)))

The anualized market volatility is 34.632% and the annualized market  returns is 20.619%


### Calculating alpha-beta and with simple linear *regression*

In [102]:
def compute_asset_info(data, rf, rm):

  # alpha & beta  and expected return
  alpha_list = []
  beta_list = []
  returns_list = []

  for name in data.drop('%5EGSPC', axis=1).columns:
    beta, alpha = np.polyfit(data['%5EGSPC'], data[name], 1)
    expected_r = rf + beta*(rm-rf)
    alpha_list.append(alpha)
    beta_list.append(beta)   
    returns_list.append(expected_r)

  asset_info = pd.DataFrame([np.array(alpha_list)*100, np.array(beta_list), np.array(returns_list)*100], columns = data.drop('%5EGSPC', axis=1).columns, index= ['alpha %','beta', 'Expected Return %'])

  return asset_info

In [103]:
df_ab = compute_asset_info(daily_returns.dropna(), Rm, rf)

In [104]:
df_ab

,NOW,AMD,JPM,MSCI,GE,RMD,BIO,APD,REG,MPC,BKNG
alpha %,0.193931,0.222666,-0.043310,0.098310,-0.082089,0.076344,0.145905,0.022820,-0.095086,-0.040886,-0.006969
beta,1.037557,1.146054,1.276287,1.141657,1.223550,0.973787,0.725837,1.031937,1.232193,1.628696,1.111597
Expected Return %,8.563619,7.302969,5.789761,7.354063,6.402531,9.304577,12.185551,8.628913,6.302101,1.695059,7.703333


### CAPM For a portfolio of assets

In [117]:
def portfolio_return(rm, rf, vol_m, vol_p):

  rp = rf + (rm-rf)*(vol_p/vol_m)

  return rp

In [118]:
returns_p = []
for vol in min_vols:
  returns_p.append(portfolio_return(Rm, rf, Sigma_m, vol))

returns_p = np.array(returns_p)

# **Theory of the Arbirtage Pricing Model:**




The arbitrage pricing theory (APT) is another theory that seeks to explain the relationship between expected return and risk.
In contrast to CAPM, APT assumes only that there are no arbitrage opportunities (same asset with two different prices <=> quoted differently on different exchanges).

The APT suggests that multiple factor can explain the expected rate of return of a risky asset: 


*   market factor
*   marco-economi factors
*   Fundamentals company attributes

However, this model doesn't explain which of these factors adds to the explanatory power of the relationship.

Fama and French proposed a 3-factor model:
$E(R_p)= \alpha_p + \beta_{p,m} [E(R_m) -rf] +  \beta_{p,smb}[E(R_{SMB})] + \beta_{p,hml}[E(R_{HML})]$

Other factors can be added such as momentum, robust minus weak (RMW). 

I suggest to read the following book about factor modeling, to have more details on how the APT model can be improved.

http://www.mlfactor.com/factor.html (chap 3)





PS: I needed more specific data to implement the APT model (ie: smal minus big data etc ..). Otherwise i would have done it.